In [1]:
import pandas as pd

data = pd.read_csv('fog_dataset.csv')

In [2]:
import numpy as np
from scipy.stats import entropy
from tqdm import tqdm 
from typing import List
EPS = 1e-12

def calculate_signal_metrics(data, window_size):
    result = {}
    
    signal = data[['AccV', 'AccML', 'AccAP']]
    
    start_hesitation = data['StartHesitation']
    turn = data['Turn']
    walking = data['Walking']
    

    num_windows = len(signal) // window_size
    
    dc_components = {}
    std_deviations = {}
    variances = {}
    entropies = {}
    energies = {}
    start_hesitations = []
    turns = []
    walkings = []
    
    for column in signal.columns:
        dc_components[column] = []
        std_deviations[column] = []
        variances[column] = []
        entropies[column] = []
        energies[column] = []
    
    for i in tqdm(range(num_windows)):
        window = signal[i*window_size:(i+1)*window_size]
        
        for column in signal.columns:
            dc_component = np.mean(window[column])
            dc_components[column].append(dc_component)
            
            std_deviation = np.std(window[column])
            std_deviations[column].append(std_deviation)
            
            variance = np.var(window[column])
            variances[column].append(variance)
            
            normalized_magnitudes = np.abs(np.fft.fft(window[column]))**2 / window_size
            signal_entropy = entropy(normalized_magnitudes + EPS)
            entropies[column].append(signal_entropy)
            
            energy = np.sum(np.abs(np.fft.fft(window[column]))**2) / window_size
            energies[column].append(energy)
        
        start_hesitation_window = start_hesitation[i*window_size:(i+1)*window_size]
        turn_window = turn[i*window_size:(i+1)*window_size]
        walking_window = walking[i*window_size:(i+1)*window_size]
        
        
        
        start_hesitations.append(max(start_hesitation_window))
        turns.append(max(turn_window))
        walkings.append(max(walking_window))
    
    result = {
        'mean': dc_components,
        'std': std_deviations,
        'var': variances,
        'entropy': entropies,
        'energy': energies,
        'StartHesitation': start_hesitations,
        'Turn': turns,
        'Walking': walkings
    }
    
    return result

In [3]:
res = calculate_signal_metrics(data.head(1400), 1)

  0%|          | 0/1400 [00:00<?, ?it/s]

100%|██████████| 1400/1400 [00:01<00:00, 811.95it/s]


In [4]:
targets = ["StartHesitation", 'Turn', "Walking"]
final = {}
for col, values in res.items():
    if col in targets:
        final[col] = values
        continue
    for name, value in values.items():
        final[col + "_" + name] = value

In [5]:
df = pd.DataFrame(final)

In [6]:
df

,mean_AccV,mean_AccML,mean_AccAP,std_AccV,std_AccML,std_AccAP,var_AccV,var_AccML,var_AccAP,entropy_AccV,entropy_AccML,entropy_AccAP,energy_AccV,energy_AccML,energy_AccAP,StartHesitation,Turn,Walking
0,-0.970018,0.061626,-0.265625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.940936,0.003798,0.070557,0,0,0
1,-0.984375,0.044497,-0.265625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.968994,0.001980,0.070557,0,0,0
2,-0.984375,0.029016,-0.265625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.968994,0.000842,0.070557,0,0,0
3,-0.984375,0.015625,-0.265625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.968994,0.000244,0.070557,0,0,0
4,-0.984670,0.015330,-0.265625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.969574,0.000235,0.070557,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,-1.250000,-0.062329,-0.202955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.562500,0.003885,0.041191,0,0,0
1396,-1.250012,-0.014704,-0.171875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.562530,0.000216,0.029541,0,0,0
1397,-1.265563,0.031066,-0.171875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.601651,0.000965,0.029541,0,0,0
1398,-1.280820,0.076662,-0.171875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.640501,0.005877,0.029541,0,0,0
